### LLM Chain 만들기


## 1. 환경 구성

### 1) 라이브러리 설치

In [ ]:
#%pip install -q langchain langchain-openai dotenv

### 2) OpenAI 인증키 설정
https://openai.com/

In [ ]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

In [ ]:
import langchain
print(langchain.__version__)

## 2. LLM Chain

### 1) Prompt + LLM


In [ ]:
from langchain_openai import ChatOpenAI

# model
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# chain 실행
result = llm.invoke("인공지능 모델의 학습 원리에 대하여 쉽게 설명해 주세요.")
print(type(result))

In [ ]:
print(type(result))
print(result)

In [ ]:
print(result.content)

### 2) PromptTemplate + LLM

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("You are an expert in AI Expert. Answer the question. <Question>: {input}에 대해 쉽게 설명해주세요.")
prompt

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# chain 연결 (LCEL)
chain = prompt | llm
print(type(chain))

# chain 호출
result = chain.invoke({"input": "인공지능 모델의 학습 원리"})
print(type(result))

In [ ]:
print(result.content)

### 3) PromptTemplate + LLM(invoke()) + StrOutputParser

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 1. 컴포넌트 정의
prompt = PromptTemplate.from_template("You are an expert in AI Expert. Answer the question. <Question>: {input}에 대해 쉽게 설명해주세요.")

llm = ChatOpenAI(model="gpt-3.5-turbo")

output_parser = StrOutputParser()

# 2. chain 생성 (LCEL)
chain = prompt | llm | output_parser
print(type(chain))

# 3. chain의 invoke 호출
result = chain.invoke({"input": "인공지능 모델의 학습 원리"})
print(type(result))

In [ ]:
print(result)

### 4) PromptTemplate + LLM(stream()) + StrOutputParser

In [ ]:

from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 1. 컴포넌트 정의
prompt = PromptTemplate.from_template("You are an expert in AI Expert. Answer the question. <Question>: {input}에 대해 쉽게 설명해주세요.")

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
lm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)

# chain 연결 (LCEL)
chain = prompt | llm | StrOutputParser()

# 스트리밍 출력을 위한 요청
answer = chain.stream({"input": "인공지능 모델의 학습 원리"})
# 스트리밍 출력
#print(answer)

for token in answer:
    # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
    print(token, end="", flush=True)

##### 2) Multiple Chains
* Multi Chain을 활용한 영화 추천 및 줄거리 요약

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Step 1: 사용자가 입력한 장르에 따라 영화 추천
prompt1 = ChatPromptTemplate.from_template("{genre} 장르에서 추천할 만한 영화를 한 편 알려주세요.")

# Step 2: 추천된 영화의 줄거리를 요약
prompt2 = ChatPromptTemplate.from_template("{movie} 추전한 영화의 제목을 먼저 알려주시고, 줄을 바꾸어서 영화의 줄거리를 3문장으로 요약해 주세요.")

# OpenAI 모델 사용
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# 체인 1: 영화 추천 (입력: 장르 → 출력: 영화 제목)
chain1 = prompt1 | llm | StrOutputParser()

# Step 1: 사용자가 입력한 장르에 따라 영화 추천
movie = chain1.invoke({"genre": "Drama"})  # 영화 제목 얻기
print(" 추천된 영화:", movie)  # movie 값 출력

In [ ]:
# 체인 2: 줄거리 요약 (입력: 영화 제목 → 출력: 줄거리)
chain2 = (
    {"movie": chain1}  # chain1의 출력을 movie 변수로 전달
    | prompt2
    | llm
    | StrOutputParser()
)
print(chain2)

# 실행: "SF" 장르의 영화 추천 및 줄거리 요약
response = chain2.invoke({"genre": "Drama"})
print("\n🔹 영화 줄거리 요약:\n", response) 

##### chain1과 chain2에서 영화 제목이 일관되게 전달 되도록 변경

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import re

# Step 1: 사용자가 입력한 장르에 따라 영화 추천
prompt1 = ChatPromptTemplate.from_template("{genre} 장르에서 추천할 만한 영화를 한 편 알려주세요.")

# Step 2: 추천된 영화의 줄거리를 요약
prompt2 = ChatPromptTemplate.from_template("{movie} 추천한 영화의 제목을 먼저 알려주시고, 줄을 바꾸어서 영화의 줄거리를 3문장으로 요약해 주세요.")

# OpenAI 모델 사용
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# 체인 1: 영화 추천 (입력: 장르 → 출력: 영화 제목)
chain1 = prompt1 | llm | StrOutputParser()

# Step 1: 사용자가 입력한 장르에 따라 영화 추천
movie_response = chain1.invoke({"genre": "Drama"})  # 영화 제목 얻기
print(" 추천된 영화 (원본 응답):", movie_response)

#  영화 제목만 추출 (첫 번째 큰따옴표 안의 문자열 찾기)
match = re.search(r'"(.*?)"', movie_response)
if match:
    movie_title = match.group(1)
else:
    movie_title = movie_response.split(" ")[0]  # 대안으로 첫 번째 단어 사용

print(" 추출된 영화 제목:", movie_title)

# 체인 2: 줄거리 요약 (입력: 영화 제목 → 출력: 줄거리)
chain2 = prompt2 | llm | StrOutputParser()

# 실행: chain1의 출력(영화 제목)을 chain2에 명확히 전달
response = chain2.invoke({"movie": movie_title})
print("\n 영화 줄거리 요약:\n", response)
